In [12]:
import sqlite3

db = sqlite3.connect('/Users/kevin/chat_isearch.db')

In [13]:
db.execute("""
attach database '/Users/kevin/chat.db' as imsg;
""")

In [14]:
db.execute("""
select
    datetime (message.date / 1000000000 + strftime ("%s", "2001-01-01"), "unixepoch", "localtime") AS message_date,
    message.date / 1000000000, -- nanosecond epoch -> seconds since epoch
	message.guid as guid,
	message.text, 
	message.is_from_me, 
	message.handle_id, 
	chat.display_name
from
	imsg.chat
	-- Find chat IDs
    JOIN chat_message_join ON imsg.chat. "ROWID" = chat_message_join.chat_id
    JOIN message ON chat_message_join.message_id = message. "ROWID"
    -- Filter by messages where we haven't calculates message embeddings.
	full outer join message_embeddings on message_embeddings.guid = message.guid 
where
	message_embeddings.embed is null
order by
	message_date desc
limit 10;
""").fetchall()

[('2023-02-18 09:50:24',
  698435424,
  '54680226-B488-4D25-A30F-B25DF156FF07',
  None,
  0,
  388,
  ''),
 ('2023-02-18 09:26:12',
  698433972,
  '983AF240-750A-402B-85CA-5CA6DB5554FF',
  None,
  1,
  415,
  ''),
 ('2023-02-18 09:24:27',
  698433867,
  '21782CDA-EF59-45B1-9A4B-05DE5C35FFE1',
  None,
  0,
  388,
  ''),
 ('2023-02-18 09:21:13',
  698433673,
  '028B2483-0E52-423B-9233-FF068CD75C30',
  None,
  1,
  415,
  ''),
 ('2023-02-18 09:20:48',
  698433648,
  '1DBD4C53-880F-42B6-88E6-4AA1A92EF36F',
  None,
  1,
  415,
  ''),
 ('2023-02-18 09:20:23',
  698433623,
  'ED839A36-66AF-45F8-8EED-524D8D95F004',
  'beat em up',
  1,
  388,
  ''),
 ('2023-02-18 09:20:11',
  698433611,
  '9AA1A782-6376-4974-8085-CB04AC3949EB',
  'Exactlyyy',
  1,
  388,
  ''),
 ('2023-02-18 09:20:07',
  698433607,
  'D46744E7-E6EE-4D25-8135-4AD81CB6C67B',
  'Loved “Lol hopefully this rollup stuff can make this not the case”',
  1,
  388,
  ''),
 ('2023-02-18 09:18:47',
  698433527,
  '35EF1468-DA1B-4A8B-B1FF-

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/kevin/.local/share/virtualenvs/isearch-Xn7T1C7t/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = model.to("mps")
model.device

In [5]:
# Throughput test

# Create a 256 byte sentence
sent = "A" * 256
sentences = [sent] * 4096 # 256kb
sentences *= 32 # 8MB

# Time the encoding for 8 MB
import time
start = time.time()
embeddings = model.encode(sentences, convert_to_tensor=True)
end = time.time()
print("Encoding time: ", end - start)

Encoding time:  72.55669522285461
